In [1]:
import sys
sys.path.append('../../../')

In [2]:
import numpy as np

from occts.classifiers.deep import ResnetClassifier, OneClassTrainer
from occts.classifiers.deep_task import OneClassTask
from occts.utils.data import load_ucr_dataset, create_datasets, create_dataloaders

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from typing import Tuple, Optional

In [3]:
DATASETS = [
    'Yoga', 'WormsTwoClass', 'Wine', 'Wafer', 'TwoLeadECG', 'Strawberry', 'SemgHandGenderCh2', 'BeetleFly',
    'BirdChicken', 'Computers', 'DistalPhalanxOutlineCorrect', 'Earthquakes', 'ECG200', 'ECGFiveDays',
    'FordA', 'FordB', 'HandOutlines', 'ItalyPowerDemand', 'MiddlePhalanxOutlineCorrect', 'Chinatown',
    'FreezerRegularTrain', 'FreezerSmallTrain', 'GunPointAgeSpan', 'GunPointMaleVersusFemale',
    'GunPointOldVersusYoung', 'PowerCons', 'Coffee', 'Ham', 'Herring', 'Lightning2', 'MoteStrain',
    'PhalangesOutlinesCorrect', 'ProximalPhalanxOutlineCorrect', 'ShapeletSim', 'SonyAIBORobotSurface1',
    'SonyAIBORobotSurface2', 'ToeSegmentation1', 'ToeSegmentation2', 'HouseTwenty'
]

In [4]:
dataset_list = []
dataset_label = []
dataset_roc_auc = []
dataset_acc = []

In [5]:
for dataset in DATASETS:
    print(f'Testing with dataset: {dataset}...')
    x_train, x_test, y_train, y_test = load_ucr_dataset(dataset=dataset)

    # Get unique labels to do the one class classification task
    unique_labels = np.unique(y_train)

    for label in unique_labels:
        print(f'\tClassifying the label {label}...')

        train_dataset, _, test_dataset = create_datasets(x_train, x_test, y_train, y_test, label)
        train_loader, _, test_loader = create_dataloaders(train_dataset=train_dataset, test_dataset=test_dataset)

        model = ResnetClassifier(in_channels=1, mid_channels=64, latent_dim=32)
        octask = OneClassTask(model=model, objective='soft-boundary', nu=0.1)

        octask.train(dataloader=train_loader, epochs=150, lr=1e-3, weight_decay=1e-6)
        octask.test(dataloader=test_loader)

        dataset_list.append(dataset)
        dataset_label.append(label)
        dataset_acc.append(octask.trainer.test_acc)
        dataset_roc_auc.append(octask.trainer.test_auc)

Testing with dataset: Yoga...
	Classifying the label 1...
			Test set AUC 0.4928245994842199
			Test set Accuracy 0.4623333333333333
	Classifying the label 2...
			Test set AUC 0.5332721925924404
			Test set Accuracy 0.5586666666666666
Testing with dataset: WormsTwoClass...
	Classifying the label 1...
			Test set AUC 0.528236914600551
			Test set Accuracy 0.4935064935064935
	Classifying the label 2...
			Test set AUC 0.3753443526170799
			Test set Accuracy 0.4805194805194805
Testing with dataset: Wine...
	Classifying the label 1...
			Test set AUC 0.5281207133058985
			Test set Accuracy 0.5
	Classifying the label 2...
			Test set AUC 0.2949245541838134
			Test set Accuracy 0.5
Testing with dataset: Wafer...
	Classifying the label -1...
			Test set AUC 0.7985347985347985
			Test set Accuracy 0.2034393251135626
	Classifying the label 1...
			Test set AUC 0.665016879350586
			Test set Accuracy 0.9104477611940298
Testing with dataset: TwoLeadECG...
	Classifying the label 1...
			Test set A

In [6]:
import pandas as pd

results = pd.DataFrame({
    'dataset': dataset_list,
    'label': dataset_label,
    'accurcy': dataset_acc,
    'roc_auc': dataset_roc_auc
})

results.to_csv('./resnet_results.csv', index=False)